# Pulling User Descriptions with Tweepy


## Important Note

This assignment requires you to have a version of Tweepy that is at least version 4. The latest version is 4.10 as I write this. Critically, this version of Tweepy is *not* on the upgrade path from Version 3, so you will not be able to simply upgrade the package if you are on Version 3. Instead you will need to explicitly install version 4, which you can do with a command like this: `pip install "tweepy>=4"`. You will also be using Version 2 of the Twitter API for this assignment. 

Run the below cell. If your version of Tweepy begins with a "4", then you should be good to go. If it begins with a "3" then run the following command, found [here](https://stackoverflow.com/questions/5226311/installing-specific-package-version-with-pip), at the command line or in a cell: `pip install -Iv tweepy==4.9`. (You may want to update that version number if Tweepy has moved on past 4.9. 

In [ ]:
pip show tweepy

In [ ]:
import tweepy
import os
import datetime
import re
from pprint import pprint


We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you in the repo. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [ ]:
from chandler_api_keys import api_key, api_key_secret, bearer_token

In [ ]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with my handle, `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [ ]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)



Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [ ]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break
    

Let's find the person who follows this handle who has the most followers. 

In [ ]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [ ]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [ ]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

Now a few questions for you about the user object.

Q: How many fields are being returned in the `response` object? 

A: <!-- Put your answer here --> 

---

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.) 

A: <!-- Put your answer here --> 

---

Q: How many friends, followers, and tweets does this user have? 

A: <!-- Put your answer here --> 


Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [ ]:

response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

In [ ]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=25, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break
        



In [ ]:
print(f"We have {len(handle_followers)} followers following {handle}.")

handle_followers[:10]

## Pulling Twitter Data for Your Handles

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [ ]:
accounts = dict()
handles = ['robynkonichiwa','cher'] # replace with yours !

for handle in handles : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    accounts[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in accounts.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")
    

Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per handle. 


In this following cells, build on the above code to pull some of the followers and their data for your two hand. As you pull the data, write the follower ids to a file called `[handle]_followers.txt`. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

In addition to creating a file that only has follower IDs in it, you will create a file that includes user data. From the response object please extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

Store the fields with one user per row in a tab-delimited text file with the name `[handle]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 

One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

In [ ]:
num_followers_to_pull = 200*1000 # feel free to use this to limit the number of followers you pull.

In [ ]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs


# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    # Create the output file names 
    
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"
    
    
    # Using tweepy.Paginator (https://docs.tweepy.org/en/latest/v2_pagination.html), 
    # use `get_users_followers` to pull the follower data requested. 

    # For each response object, extract the needed fields and store them in a dictionary or
    # data frame. 

    # I recommend writing your results for every response. This isn't the most efficient option
    # (since you're opening and closing the file regularly), but it ensures that your 
    # work is saved in case there is an issue with the API connection. 
    
    # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response


        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)


In [ ]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark.
    you only run for the border
    when you see the whole city
    running as well.

"""
# This kind of description won't work in a tab-delimited text file.
# We haven't learned about regular expressions yet, but the below
# lines will clean up these tricky descriptions

clean_description = re.sub(r"\s+"," ",tricky_description).strip()
clean_description